In [ ]:
import pandas as pd
import numpy as np
import yaml

from sqlalchemy import create_engine
import psycopg2

In [ ]:
def crear_viaje_id_acumulada(dt,ventana_minutos=120):
 
    dt.delta = dt.delta.shift(1)
    dt['delta']=(pd.to_timedelta(dt.delta) / np.timedelta64(1, 'm')).fillna(0)

    cumulativa = 0
    viaje_id = 0
    viajes = []
    for i in dt.delta:
        cumulativa += i
        if cumulativa < ventana_minutos:
            pass
        else:
            cumulativa = 0
            viaje_id += 1
        viajes.append(viaje_id)
    dt['viaje_id'] = viajes

    return dt

def consolidar_tabla_viajes(dt):
    # si alguno de los destinos de los tramos es nulo
    # el destino del viaje es nulo
    if any(dt.h3_d.isnull()):
        viaje = pd.DataFrame({
                      'tarjeta':dt.tarjeta.unique(),
                      'viaje_id':dt.viaje_id.unique(),
                      'linea':' - '.join(dt.linea),
                      'fecha':dt.fecha.iloc[0],
                      'tramos':len(dt.tramo_id),
                      'lat_o':dt.lat_o.iloc[0],
                      'lon_o':dt.lon_o.iloc[0],
                      'h3_o':dt.h3_o.iloc[0],
                      'lat_d':np.nan,
                      'lon_d':np.nan,
                      'h3_d':np.nan
                     },index=[0])
    else:
        viaje = pd.DataFrame({
                      'tarjeta':dt.tarjeta.unique(),
                      'viaje_id':dt.viaje_id.unique(),
                      'linea':' - '.join(dt.linea),
                      'fecha':dt.fecha.iloc[0],
                      'tramos':len(dt.tramo_id),
                      'lat_o':dt.lat_o.iloc[0],
                      'lon_o':dt.lon_o.iloc[0],
                      'h3_o':dt.h3_o.iloc[0],
                      'lat_d':dt.lat_d.iloc[-1],
                      'lon_d':dt.lon_d.iloc[-1],
                      'h3_d':dt.h3_d.iloc[-1],
                     },index=[0])
    return viaje

In [ ]:
DB_USERNAME = 'sube_user'
DB_PASSWORD = 'sube_pass'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'sube'
DB_SCHEMA = 'public'

In [ ]:
# Conectar a la db
conn = psycopg2.connect(user = DB_USERNAME,
                                      password = DB_PASSWORD,
                                      host = DB_HOST,
                                      port = DB_PORT,
                                      database = DB_NAME)

In [ ]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'
    .format(DB_USERNAME, DB_PASSWORD, DB_HOST,
            DB_PORT, DB_NAME))

In [ ]:
cantidad_de_tarjetas = 6000

In [ ]:
# traer tarjetas que tengan vacio el viaje_id
# traer trx de tarjetas que hayan pasado por la B SIN MISSING

q = """
select *
from tramos_linea_b
where tarjeta in (
    select distinct tarjeta 
    from tramos_linea_b tlb 
    where viaje_id is null
    limit %i
)
order by tarjeta, tramo_id
"""%cantidad_de_tarjetas

tramos = pd.read_sql(q, conn)

In [ ]:
tramos

In [ ]:
# popular en tabla tramos el viaje_id
%time tramos_con_viaje_id = tramos.groupby(['tarjeta']).apply(crear_viaje_id_acumulada)

In [ ]:
tramos_con_viaje_id

In [ ]:
# actualizar tabla de tramos en db con viaje_id
tramos_con_viaje_id.to_sql('viajes_tmp', engine, schema=DB_SCHEMA,index=False,method='multi')

In [ ]:
update_viaje_id_query = """
UPDATE tramos_linea_b 
SET viaje_id = v.viaje_id
FROM viajes_tmp v
WHERE tramos_linea_b.tarjeta = v.tarjeta
and tramos_linea_b.tramo_id = v.tramo_id;


DROP TABLE IF EXISTS viajes_tmp; 
"""

In [ ]:
cur = conn.cursor()
cur.execute(update_viaje_id_query)
cur.close()
conn.commit()

In [ ]:
# crear tabla de viajes
tabla_viajes = tramos_con_viaje_id.groupby(['tarjeta','viaje_id'],as_index=False)\
    .apply(consolidar_tabla_viajes).reset_index(drop=True)
tabla_viajes

In [ ]:
# appendear viajes a tabla de viajes 

In [ ]:
tabla_viajes.to_sql('viajes_linea_b', engine, schema=DB_SCHEMA,index=False,method='multi',if_exists = 'append')

In [ ]:
q="""
select count(distinct tarjeta) 
from tramos_linea_b
where viaje_id  is null
"""
quedan = pd.read_sql(q, conn)
quedan = quedan.iloc[0,0]

In [ ]:
print('Quedan %s tarjetas'%quedan)